# Observer Coding Exercise

Imagine a game where one or more rats can `attack` a player. Each individual rat has an initial `attack` value of 1. However, rats `attack` as a swarm, so each rat's attack value is actually equal to the total number of rats in play.

Given that a rat enters play through the initializer and leaves play (dies) via its `__exit__` method, please implement the Game and Rat classes so that, at any point in the game, the Attack value of a rat is always consistent.

Here's a sample unit test your code should pass:

```python
def test_three_rats_one_dies(self):
    game = Game()
 
    rat = Rat(game)
    self.assertEqual(1, rat.attack)
 
    rat2 = Rat(game)
    self.assertEqual(2, rat.attack)
    self.assertEqual(2, rat2.attack)
 
    with Rat(game) as rat3:
        self.assertEqual(3, rat.attack)
        self.assertEqual(3, rat2.attack)
        self.assertEqual(3, rat3.attack)
```

In [1]:
from unittest import TestCase

In [2]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

class Game:
    def __init__(self):
        self.rat_enters = Event()
        self.rat_dies = Event()
        self.notify_rat = Event()

In [3]:
class Rat:
    def __init__(self, game):
        self.game = game
        self.attack = 1

        game.rat_enters.append(self.rat_enters)
        game.notify_rat.append(self.notify_rat)
        game.rat_dies.append(self.rat_dies)

        self.game.rat_enters(self)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.game.rat_dies(self)

    def rat_enters(self, which_rat):
        if which_rat != self:
            self.attack += 1
            self.game.notify_rat(which_rat)

    def notify_rat(self, which_rat):
        if which_rat == self:
            self.attack += 1

    def rat_dies(self, which_rat):
        self.attack -= 1

In [4]:
game = Game()
rat = Rat(game)
print(rat.attack) # 1

1


In [5]:
game = Game()
rat = Rat(game)
rat2 = Rat(game)
print(rat.attack) # 2
print(rat2.attack) # 2

2
2


In [6]:
game = Game()

rat = Rat(game)
print('== 1 rat ==')
print(rat.attack) # 1

rat2 = Rat(game)
print('== 2 rats ==')
print(rat.attack) # 2
print(rat2.attack) # 2

# this will call enter and exit
print('== 3 rats ==')
with Rat(game) as rat3:
    print(rat.attack) # 3
    print(rat2.attack) # 3
    print(rat3.attack) # 3

# equivalent to ...
# rat3 = Rat(game)
# print(rat.attack) # 3
# print(rat2.attack) # 3
# print(rat3.attack) # 3
# rat3.__exit__(None, None, None)

print('== 2 rats ==')
print(rat.attack) # 2
print(rat2.attack) # 2


== 1 rat ==
1
== 2 rats ==
2
2
== 3 rats ==
3
3
3
== 2 rats ==
2
2
